In [ ]:
import os

# fetch data
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

# process data
import pandas as pd
from sklearn.model_selection import train_test_split

# model data
import xgboost as xgb

# visualize data
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# evals
from sklearn.metrics import root_mean_squared_error

from dotenv import load_dotenv
load_dotenv('.env')

In [ ]:
DB='snowflake_sample_data'
SCHEMA='tpch_sf1'

In [ ]:
engine = create_engine(
    URL(
        user=os.environ["SNOWFLAKE_USER"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        account=os.environ["SNOWFLAKE_ACCOUNT_IDENTIFIER"],
        warehouse=os.environ["SNOWFLAKE_WAREHOUSE"],
        database=DB,
        schema=SCHEMA
    )
)

## Run sample query

In [ ]:
# https://docs.snowflake.com/en/user-guide/sample-data-tpch
query = """
select
       l_returnflag,
       l_linestatus,
       sum(l_quantity) as sum_qty,
       sum(l_extendedprice) as sum_base_price,
       sum(l_extendedprice * (1-l_discount)) as sum_disc_price,
       sum(l_extendedprice * (1-l_discount) * (1+l_tax)) as sum_charge,
       avg(l_quantity) as avg_qty,
       avg(l_extendedprice) as avg_price,
       avg(l_discount) as avg_disc,
       count(*) as count_order
 from
       snowflake_sample_data.tpch_sf1.lineitem
 where
       l_shipdate <= dateadd(day, -90, to_date('1998-12-01'))
 group by
       l_returnflag,
       l_linestatus
 order by
       l_returnflag,
       l_linestatus;
"""
df = pd.read_sql(query, engine)

In [ ]:
df

## Fetch unprocessed data for feature engineering

In [ ]:
feature_columns = ['l_partkey','l_suppkey','l_quantity','l_discount','l_tax']
prediction_column = 'l_extendedprice'
all_columns = feature_columns + [prediction_column]

In [ ]:
N=1_000_000

In [ ]:
all_data = pd.read_sql(f'select {", ".join(all_columns)} from lineitem limit {N}', engine)

In [ ]:
all_data.describe()

In [ ]:
sns.pairplot(all_data)

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(
    all_data[feature_columns], 
    all_data[prediction_column], 
    test_size=0.05
)

X_train, X_validation, y_train, y_validation = train_test_split(X_train,  y_train, test_size=0.2)

print(f"Training data: {X_train.shape[0]} rows")
print(f"Validation data: {X_validation.shape[0]} rows")
print(f"Holdout data: {X_holdout.shape[0]} rows")

In [ ]:
model = xgb.XGBRegressor(
    eval_metric='rmse',
)
model.fit(X_train, y_train, eval_set=[(X_validation, y_validation)], verbose=0)
booster = model.get_booster()
booster.feature_names = feature_columns

In [ ]:
y_hat = model.predict(X_holdout)

In [ ]:
root_mean_squared_error(y_holdout, y_hat)